## Things you need
+ config file with api key
+ Your user agent info can get https://www.whatismybrowser.com/detect/what-is-my-user-agent and replace the headers variable with your info
    + header variable is line 10 in both scraping block
+ Script splits up your sources into 2 queries and saves them as csv files.  I created a csv folder for them

## change query
Hit other candidates as well and make sure that the range (20,31) is no more than 30 days ago.  If you get a out of range error just change 20 to whatever day of month it is, or day of month +1

In [1]:
import pandas as pd
import config
import math
import csv
import sys
from newsapi import NewsApiClient
import newspaper
import requests
from newspaper import fulltext
import time
# Hit Api with credentials
newsapi = NewsApiClient(api_key=config.api_key)


## global variables
candidates_list = []
total_count = []


def error_checks():
    ##validate that df shape[0] is equal to expected query count
    print("we should expect: {} articles".format(sum(total_count)))
    print("we have: {} articles".format(Cand_df.shape[0]))


    ## test 100 results theory
    over_100 = sum([x-100 for x in total_count if x>100 ])
    print("{} results came into query that exceeded 100 hits in a day.".format(over_100))


    ##unique links
    unique_array = Cand_df.url.unique()
    print("we have {} unique links".format(unique_array.shape[0]))
    
    duplicateRowsDF = Cand_df[Cand_df.duplicated("url")]
    print("we have duplciate shape df of: {} ".format(duplicateRowsDF.shape))

def clean_query(query):
    for x in query['articles']:
        try:
            x["source"] = x["source"]["name"]
        except:
            pass
        try:
            x['publishedAt'] = str.split(x['publishedAt'], "T")[0]
        except:
            pass
        try:
            del x['urlToImage']
        except KeyError:
            pass
    my_df = pd.DataFrame(query["articles"])
    return my_df

def hit_api(start, end, q, myString2):

    # catch bug with formatted strings for dates
    if end < 10:
        start_str = "0" + str(start)
        end_str = "0" + str(end)
    elif end == 10:
        start_str = "0" + str(start)
        end_str = str(end)
    else:
        start_str = str(start)
        end_str = str(end)

    # API query
    all_articles = newsapi.get_everything(q=q,
                                          sources=myString2,
                                          language='en',
                                          from_param='2019-09-{}'.format(
                                              start_str),
                                          to='2019-09-{}'.format(end_str),
                                          sort_by='relevancy',
                                          page_size=100,
                                          page=1)

    # get count
    total_pages = math.ceil(all_articles["totalResults"]/100)
    print("query will return: " + str(all_articles["totalResults"]))

    # store count to check versus dimension of df later
    total_count.append(all_articles["totalResults"])

    # Clen query
    all_articles = clean_query(all_articles)

    # append to list
    candidates_list.append(all_articles)
    return(candidates_list)


In [2]:

## import data dictionary form github
url = 'https://raw.githubusercontent.com/mburke65/Data_698_Final/master/lists.csv'
df = pd.read_csv(url, error_bad_lines=False)

# picking data sources from file on github
a= df.iloc[[16,17,24],:]
b= df.iloc[[40,43,77],:]
list_sources = a["sources"].tolist()

# build out string for query request
myString = ",".join(list_sources)

list_sources2 = b["sources"].tolist()
myString2 = ",".join(list_sources2)



## Scrape Wall street journal and NYT

In [3]:
for x in range(22, 31):
    df = hit_api(x, x+1, 'Warren OR Elizabeth Warren', myString2)

# collapse list on itself to build big df
Cand_df = pd.concat(df)
Cand_df = Cand_df.reset_index(drop=True)
error_checks()


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'}
## Build counts as check on progress of loop. will count down Url left to hit
counts=Cand_df['url'].shape[0]
list_full_text=[]
for link in Cand_df['url']:
    counts-=1
    print(counts)
    html = requests.get(link,headers=headers).text
    try: text = fulltext(html)
    except: 
        print("no words found")
        text="no words found"
        list_full_text.append(text)
        continue
    list_full_text.append(text)
Cand_df['full_art']=pd.Series(list_full_text)
Cand_df.to_csv('data_csvs/data1'+str(time.time())+'.csv')

query will return: 21
query will return: 30
query will return: 25
query will return: 19
query will return: 21
query will return: 14
query will return: 5
query will return: 17
query will return: 324
we should expect: 476 articles
we have: 252 articles
224 results came into query that exceeded 100 hits in a day.
we have 179 unique links
we have duplciate shape df of: (73, 7) 
251
250
249
248
no words found
247
246
245
244
243
242
241
no words found
240
239
238
237
236
235
234
233
232
231
230
229
228
227
226
225
no words found
224
223
222
221
220
no words found
219
218
217
216
215
214
213
no words found
212
211
210
209
208
207
206
205
204
203
202
201
200
199
198
197
196
195
no words found
194
no words found
193
192
191
190
189
188
187
186
185
184
183
182
181
180
179
178
177
176
175
174
173
172
171
no words found
170
169
168
167
166
165
164
163
162
161
160
159
158
157
156
155
154
153
152
151
150
149
148
147
146
145
144
143
142
141
140
139
138
137
136
135
134
133
132
131
130
129
128
127
126

## Scrape WAPO New York magazine


In [4]:
for x in range(22, 31):
    df = hit_api(x, x+1, 'Biden OR Elizabeth Warren', myString)

# collapse list on itself to build big df
Cand_df = pd.concat(df)
Cand_df = Cand_df.reset_index(drop=True)
error_checks()


## Build counts as check on progress of loop. will count down Url left to hit
counts=Cand_df['url'].shape[0]
list_full_text=[]
for link in Cand_df['url']:
    counts-=1
    print(counts)
    html = requests.get(link,headers=headers).text
    try: text = fulltext(html)
    except: 
        print("no words found")
        text="no words found"
        list_full_text.append(text)
        continue
    list_full_text.append(text)
Cand_df['full_art']=pd.Series(list_full_text)
Cand_df.to_csv('data_csvs/data1'+str(time.time())+'.csv')

query will return: 29
query will return: 26
query will return: 29
query will return: 26
query will return: 21
query will return: 23
query will return: 20
query will return: 19
query will return: 370
we should expect: 1039 articles
we have: 545 articles
494 results came into query that exceeded 100 hits in a day.
we have 385 unique links
we have duplciate shape df of: (160, 7) 
544
543
542
541
no words found
540
539
538
537
536
535
534
no words found
533
532
531
530
529
528
527
526
525
524
523
522
521
520
519
518
no words found
517
516
515
514
513
no words found
512
511
510
509
508
507
506
no words found
505
504
503
502
501
500
499
498
497
496
495
494
493
492
491
490
489
488
no words found
487
no words found
486
485
484
483
482
481
480
479
478
477
476
475
474
473
472
471
470
469
468
467
466
465
464
no words found
463
462
461
460
459
458
457
456
455
454
453
452
451
450
449
448
447
446
445
444
443
442
441
440
439
438
437
436
435
434
433
432
431
430
429
428
427
426
425
424
423
422
421
420


In [5]:
my_df=pd.read_csv('Vikas.csv')

In [8]:
my_df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'author', 'content', 'description',
       'publishedAt', 'source', 'title', 'url', 'full_art'],
      dtype='object')

In [10]:
my_df["full_art"]

0       “Young people are constantly signaling what’s ...
1       WASHINGTON — For months this spring and summer...
2       WASHINGTON — President Trump pressed the Ukrai...
3       WASHINGTON — President Trump on Saturday dismi...
4       Though he has yet to call for impeachment proc...
5       Why is this coming up now?\r\r\n\r\r\nAn intel...
6       CEDAR RAPIDS, Iowa — Former Vice President Jos...
7       “Some government lawyer would be nervous to do...
8       Will this fine-tuning be enough in an agitated...
9       President Trump in a July phone call repeatedl...
10      Kamala Harris’s campaign says she is focusing ...
11      President Trump pressed on with calls for an i...
12      WASHINGTON — Everyone here is keyed up for the...
13      To the Editor:\r\r\n\r\r\nRe “Claim on Trump I...
14      WASHINGTON—Elizabeth Warren is building a broa...
15      Presidents these days always seem to get worse...
16      On Thursday, Mr. Atkinson appeared before a me...
17      Mayor 